In [1]:
!pip install -q flwr

import flwr as fl
import tensorflow as tf
from google.colab import drive
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [2]:
CLIENT_ID = 1

drive.mount('/content/drive', force_remount=True)

DRIVE_DATA_PATH = '/content/drive/Shareddrives/FedSynDrive/Data'
data_path = os.path.join(DRIVE_DATA_PATH, f'client_{CLIENT_ID}_data.npz')

print(f"--- This notebook is now CLIENT #{CLIENT_ID} ---")
print(f"Loading data from: {data_path}")

with np.load(data_path) as data:
    x_train = data['x'].astype(np.float32)
    y_train = data['y']
    if x_train.ndim == 3:
        x_train = np.expand_dims(x_train, axis=3)

print(f"Successfully loaded {len(x_train)} samples and labels for this client.")

Mounted at /content/drive
--- This notebook is now CLIENT #1 ---
Loading data from: /content/drive/Shareddrives/FedSynDrive/Data/client_1_data.npz
Successfully loaded 20000 samples and labels for this client.


In [3]:
def create_cnn_model():
    """This must have the exact same architecture as the server's model."""
    model = Sequential([
        Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
        MaxPool2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(5, 5), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

class CnnClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train):
        self.model = model
        self.x_train = x_train
        self.y_train = y_train

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.x_train, self.y_train, verbose=0)
        return loss, len(self.x_train), {"accuracy": accuracy}

In [5]:
print(f"\n--- Starting Client #{CLIENT_ID} ---")

model = create_cnn_model()
client = CnnClient(model, x_train, y_train)

fl.client.start_numpy_client(
    server_address="35.184.5.50:8888",
    client=client
)

print(f"\n--- Client #{CLIENT_ID} has finished. ---")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), 


--- Starting Client #1 ---


INFO :      
INFO:flwr:
INFO :      Received: train message 53630157-6b10-4d50-a1e3-527270606279
INFO:flwr:Received: train message 53630157-6b10-4d50-a1e3-527270606279
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: evaluate message 0f628f1d-b07e-4986-8c02-b3a0fb479eb6
INFO:flwr:Received: evaluate message 0f628f1d-b07e-4986-8c02-b3a0fb479eb6
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: train message 7875003f-b6ab-403c-9b1e-8f835cff8ad0
INFO:flwr:Received: train message 7875003f-b6ab-403c-9b1e-8f835cff8ad0
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: evaluate message aa75dfeb-6e2d-4e72-9634-e3a92b64a37e
INFO:flwr:Received: evaluate message aa75dfeb-6e2d-4e72-9634-e3a92b64a37e
INFO :      Sent reply
INFO:flwr:Sent reply
INFO :      
INFO:flwr:
INFO :      Received: train message f0825bbb-4119-4a06-bb7f-2aaa759a46fb
INFO:flwr:Received: train message f0825bbb-


--- Client #1 has finished. ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
